<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg"
     align="right"
     width="30%"
     alt="Dask logo">

DataFrames on a Cluster
=======================

<img src="https://pandas.pydata.org/static/img/pandas.svg"
     align="left"
     width="30%"
     alt="Pandas logo">



In [1]:
#This notebook needs the [gcsfs library](https://gcsfs.readthedocs.io).
!pip install gcsfs ipywidgets

  Using cached gcsfs-2022.8.2-py2.py3-none-any.whl (25 kB)
  Using cached ipywidgets-8.0.2-py3-none-any.whl (134 kB)
  Using cached google_cloud_storage-2.5.0-py2.py3-none-any.whl (106 kB)
  Using cached google_auth_oauthlib-0.5.3-py2.py3-none-any.whl (19 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.8/169.8 kB 4.3 MB/s eta 0:00:00a 0:00:01
  Using cached widgetsnbextension-4.0.3-py3-none-any.whl (2.0 MB)
  Using cached jupyterlab_widgets-3.0.3-py3-none-any.whl (384 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached rsa-4.9-py3-none-any.whl (34 kB)
  Using cached cachetools-5.2.0-py3-none-any.whl (9.3 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached google_api_core-2.10.1-py3-none-any.whl (115 kB)
  Using cached google_resumable_media-2.3.3-py2.py3-none-any.whl (76 kB)
  Using cached google_cloud_core-2.3.2-py2.py3-none-any.whl (29 kB)
  Using cached googleapis_common_protos-1.56.4-py2.py3-none-any.whl (211 

## Read single dataframe from the cloud with Pandas

In [2]:
from gcsfs import GCSFileSystem
gcs = GCSFileSystem(token='anon')

gcs.ls('anaconda-public-data/nyc-taxi/csv/2015/')

['anaconda-public-data/nyc-taxi/csv/2015/green_tripdata_2015-01.csv',
 'anaconda-public-data/nyc-taxi/csv/2015/green_tripdata_2015-02.csv',
 'anaconda-public-data/nyc-taxi/csv/2015/green_tripdata_2015-03.csv',
 'anaconda-public-data/nyc-taxi/csv/2015/green_tripdata_2015-04.csv',
 'anaconda-public-data/nyc-taxi/csv/2015/green_tripdata_2015-05.csv',
 'anaconda-public-data/nyc-taxi/csv/2015/green_tripdata_2015-06.csv',
 'anaconda-public-data/nyc-taxi/csv/2015/yellow_tripdata_2015-01.csv',
 'anaconda-public-data/nyc-taxi/csv/2015/yellow_tripdata_2015-02.csv',
 'anaconda-public-data/nyc-taxi/csv/2015/yellow_tripdata_2015-03.csv',
 'anaconda-public-data/nyc-taxi/csv/2015/yellow_tripdata_2015-04.csv',
 'anaconda-public-data/nyc-taxi/csv/2015/yellow_tripdata_2015-05.csv',
 'anaconda-public-data/nyc-taxi/csv/2015/yellow_tripdata_2015-06.csv',
 'anaconda-public-data/nyc-taxi/csv/2015/yellow_tripdata_2015-07.csv',
 'anaconda-public-data/nyc-taxi/csv/2015/yellow_tripdata_2015-08.csv',
 'anaconda-p

In [3]:
import pandas as pd

with gcs.open('anaconda-public-data/nyc-taxi/csv/2015/yellow_tripdata_2015-01.csv') as f:
    df = pd.read_csv(f, nrows=5, parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0,0.3,16.30


## Parallelize Pandas with Dask.dataframe


In [5]:
import dask
from dask.distributed import Client, progress
client = Client('localhost:40455')
client

<Client: 'tcp://127.0.0.1:40455' processes=4 threads=8, memory=16.00 GiB>

In [6]:
import dask.dataframe as dd

df = dd.read_csv('gcs://anaconda-public-data/nyc-taxi/csv/2015/yellow_*.csv',
                 storage_options={'token': 'anon'}, 
                 parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

In [7]:
df = df.persist()
progress(df)

VBox()


Dask DataFrames
---------------

*  Coordinate many Pandas DataFrames across a cluster
*  Faithfully implement a subset of the Pandas API
*  Use Pandas under the hood (for speed and maturity)

In [8]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
npartitions=353,,,,,,,,,,,,,,,,,,,
,int64,datetime64[ns],datetime64[ns],int64,float64,float64,float64,int64,object,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [9]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                   int64
trip_distance                   float64
pickup_longitude                float64
pickup_latitude                 float64
RateCodeID                        int64
store_and_fwd_flag               object
dropoff_longitude               float64
dropoff_latitude                float64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
dtype: object

In [10]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30


In [ ]:
%time len(df)

In [ ]:
%time df.passenger_count.sum().compute()

In [ ]:
# Compute average trip distance grouped by passenger count
df.groupby(df.passenger_count).trip_distance.mean().compute()

### Tip Fraction, grouped by day-of-week and hour-of-day

In [ ]:
df2 = df[(df.tip_amount > 0) & (df.fare_amount > 0)]
df2['tip_fraction'] = df2.tip_amount / df2.fare_amount

In [ ]:
# Group df.tpep_pickup_datetime by dayofweek and hour
dayofweek = df2.groupby(df2.tpep_pickup_datetime.dt.dayofweek).tip_fraction.mean() 
hour = df2.groupby(df2.tpep_pickup_datetime.dt.hour).tip_fraction.mean()

dayofweek, hour = dask.persist(dayofweek, hour)
progress(dayofweek, hour)

### Plot results

This requires matplotlib to be installed

In [ ]:
%matplotlib inline

In [ ]:
hour.compute().plot(figsize=(10, 6), title='Tip Fraction by Hour')